# Day 4: Scratchcards

Again, I'm creating a data class for the elements of the puzzle (today `Card`). This time I put the creation of the element in a factory method in the class itself.

In [ ]:
data class Card(val id: Int, val winningNumbers: List<Int>, val myNumbers: List<Int>) {
    val wins = myNumbers.intersect(winningNumbers).size

    companion object {
        private val cardRegex = Regex("(\\d+)")

        fun fromLine(line: String): Card {
            val (id, winningNumbers, myNumbers) = line.split(':', '|')
                .map { cardRegex.findAll(it).map { it.value.toInt() }.toList() }
            return Card(id[0], winningNumbers, myNumbers)
        }
    }
}

Reading the input:

In [ ]:
import kotlin.io.path.Path
import kotlin.io.path.readLines

val cards = Path("input.txt").readLines()
    .map(Card::fromLine)

## Part 1
Bit shifting does essentially the same as using `pow` here but avoids using double or float. We also use the fact that shifting left by a negative amount results in a negative number.

In [ ]:
cards.sumOf { max(0, 1.shl(it.wins - 1)) }

## Part 2
Here I use card IDs as list index - 1.

My initial solution used two nested for loops, summing up the individual card counts in the list first before calculating the overall sum of cards.

In [ ]:
val cardCounts = MutableList(cards.size) { 1 }

for (card in cards) {
    for (i in 0 until card.wins) {
        cardCounts[card.id + i] += cardCounts[card.id - 1]
    }
}

cardCounts.sum()

Then I was curious to see if I could do it with lambdas. So here is a second version utilizing `fold`, which calculates the overall sum and reduces the list of card counts on the go.

In [ ]:
cards.fold(0 to MutableList(cards.size) { 1 }) { (sum, cardCounts), card ->
    val currentCardCount = cardCounts.removeFirst()
    (0 until card.wins).forEach { cardCounts[it] += currentCardCount }
    sum + currentCardCount to cardCounts
}.first